In [ ]:
%pip install nbdt

In [1]:
from nbdt.model import SoftNBDT
from nbdt.model import HardNBDT
from nbdt.models import ResNet18, wrn28_10_cifar10, wrn28_10_cifar100, wrn28_10  # use wrn28_10 for TinyImagenet200
from torchvision import transforms
from nbdt.utils import DATASET_TO_CLASSES, load_image_from_path, maybe_install_wordnet
from IPython.display import display
import foolbox as fb
import torch
import torchvision
#import advertorch as at
from PIL import Image

not enough values to unpack (expected 2, got 0)


In [3]:
model = wrn28_10_cifar10()
#model.linear = torch.nn.Linear(in_features=512, out_features=200, bias=True)
model = HardNBDT(
  pretrained=True,
  dataset='CIFAR10',
  arch='wrn28_10_cifar10',
  hierarchy='wordnet',
  model=model)

transforms_c = transforms.Compose([
  transforms.Resize(32),
  transforms.CenterCrop(32),
  transforms.ToTensor(),
  transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

Downloading: "https://github.com/alvinwan/neural-backed-decision-trees/releases/download/0.0.1/ckpt-CIFAR10-wrn28_10_cifar10-wordnet-SoftTreeSupLoss.pth" to /Users/wesley/.cache/torch/checkpoints/ckpt-CIFAR10-wrn28_10_cifar10-wordnet-SoftTreeSupLoss.pth


In [5]:
# Import ResNet18 pretrained on TinyImageNet. Wrapping with HardNBDT.
model = ResNet18()
model.linear = torch.nn.Linear(in_features=512, out_features=200, bias=True)
model = HardNBDT(
  pretrained=True,
  dataset='TinyImagenet200',
  arch='ResNet18',
  hierarchy='wordnet',
  model=model)

transforms_t = transforms.Compose([
  transforms.Resize(32),
  transforms.CenterCrop(32),
  transforms.ToTensor(),
  transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262]),
])

In [ ]:
test_0 = load_image_from_path('./test_0.jpeg')
display(test_0)

In [ ]:
test_1 = load_image_from_path('./test_1.jpeg')

In [ ]:
def tree_pred(img):
    x = transforms(img)[None]
    outputs = model.forward_with_decisions(x)
    print(outputs[1])

In [ ]:
x = transforms(test_0)[None]
outputs = model.forward_with_decisions(x)
_, predicted = outputs[0].max(1)
#cls = DATASET_TO_CLASSES['CIFAR10'][predicted[0]]

In [ ]:
test_1 = load_image_from_path('./test_1.jpeg')
display(test_1)

In [ ]:
tree_pred(test_0)

In [ ]:
import advertorch as at
gradient_model = at.attacks.GradientSignAttack(model)

In [ ]:
tree_pred(test_1, gradient_model)

In [ ]:
test_1

In [ ]:
from PIL import Image, ImageFilter  
test_1 = test_1.filter(ImageFilter.GaussianBlur(radius = 2)) 

In [ ]:
display(test_1)

In [ ]:
tree_pred(test_1)